In [ ]:
import pandas as pd
import json
# נעזרתי המון בצ'אט ג'י פי טי כי לא הרגשתי שאני מכירה את החומר טוב אבל בהחלט הבנתי כאשר אני כותבת את הקוד
#1
df_1 = pd.read_csv('bipper_users.csv')
# create a sample DataFrame
with open('bipper_activity_details.json') as f:
    social_data = json.load(f)
records = []
for entry in social_data:
    for message in entry['messages']:
        records.append({
            'user_id': entry['user_id'],
            'to_user': message['to_user'],
            'timestamp': message['timestamp'],
            'friends_count': len(entry['friends']),
            'message': message['content']
        })
df_messages = pd.DataFrame(records)
df_messages
merged_df = pd.merge(df_1, df_messages, on='user_id', how='inner')
merged_df.head(5)

#2
rows, cols = merged_df.shape
# print(f'Number of rows: {rows}, Number of columns: {cols}')

#3
merged_df['join_date'] = pd.to_datetime(merged_df['join_date'])
# מציאת המשתמש החדש ביותר
newest_user = merged_df.loc[merged_df['join_date'].idxmax()]
# print(newest_user.user_id)

#4
# הגדרת user_id של המשתמש שברצונך לבדוק
user_id_to_check = 2
#Counting the messages sent by the user
message_count = df_messages[df_messages['user_id'] == user_id_to_check].shape[0]
# print(f'User {user_id_to_check} sent {message_count} messages.')

#5
# מילה לחיפוש
word_to_find = "Let's meet"
# # פילטר כדי למצוא את השורות שמכילות את המילה
filtered_df = merged_df[merged_df['message'].str.contains(word_to_find, na=False)]
# print(filtered_df)
merged_df

#6
# ספירת ההודעות לכל משתמש
message_counts = merged_df['user_id'].value_counts().reset_index()
message_counts.columns = ['user_id', 'message_count']
# מיזוג עם נתוני החברים
user_data = pd.merge(merged_df, message_counts, on='user_id', how='left')
# מילוי ערכים חסרים בהודעות
user_data['message_count'].fillna(0, inplace=True)
# חישוב יחס ההודעות לחברים
user_data['message_per_friend'] = user_data['message_count'] / user_data['friends_count']
# מציאת המשתמש עם יחס ההודעות הנמוך ביותר
least_active_user = user_data.loc[user_data['message_per_friend'].idxmin()]
# print(least_active_user)

#7
user_counts = merged_df.groupby('user_id')['to_user'].nunique().reset_index()

# שמירת המשתמשים ששולחים הודעות רק לאדם אחד
single_recipient_users = user_counts[user_counts['to_user'] == 1]

# מיזוג עם DataFrame המקורי כדי לקבל מידע נוסף
result_df = pd.merge(merged_df, single_recipient_users[['user_id']], on='user_id', how='inner')

print(result_df)

,user_id,name,age,gender,city,join_date
0,1,Ian1,51,M,Seattle,2024-03-24
1,2,Charlie2,51,F,Atlanta,2024-02-14
2,3,Bob3,31,M,Houston,2024-08-23
3,4,Alice4,34,M,Los Angeles,2024-04-23
4,5,David5,50,F,Austin,2024-08-05
...,...,...,...,...,...,...
995,996,Bob996,44,M,New York,2024-08-06
996,997,Mark997,39,F,Atlanta,2024-09-29
997,998,Judy998,29,M,San Francisco,2024-04-11
998,999,Bob999,55,F,New York,2024-03-05
